<a href="https://colab.research.google.com/github/namtoptall/DataScience/blob/main/iception_milestone_project_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Xây dựng hệ thống gợi ý hoa

## Huấn luyện mô hình : Model training
**Lưu ý ⚠ em huấn luyện mô hình trên google colab nên nếu muốn chạy lại toàn bộ thì cần set up những tài khoản sau :**
> tài khoản kaggle và kaggle API cho việc download bộ dữ liệu

> tài khoản google Drive để kết nối với kho dữ liệu cá nhân



### kết nối với google drive và kaggle

Với kaggle : thực hiện tải tệp kaggle.json(Chứa thông tin tài khoản) lên

In [ ]:
from google.colab import files
files.upload()  # Thực hiện tải tệp kaggle.json lên từ máy tính (phải có tài khoản kaggle nha)

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Mount google drive content vào trong notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!kaggle datasets download -d namdz242221/dsp305x-01-a-vn-dataset

unzip the dataset

In [ ]:
!unzip dsp305x-01-a-vn-dataset.zip -d /content/dataset


In [ ]:
!ls /content/dataset

check content trong tệp

In [ ]:
!ls /content/dataset

In [ ]:
import os
dataset_dir = "/content/dataset"
for root, dirs, files in os.walk(dataset_dir):
    print(root, "contains", len(files), "files.")


check shape của từng ảnh

In [ ]:
from PIL import Image
import os
from collections import defaultdict

def check_unique_shapes(dataset_dir):
    # count the number of difference shapes
    shape_counts = defaultdict(int)

    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                try:
                    with Image.open(img_path) as img:
                        shape = img.size  # Lấy kích thước (chiều rộng, chiều cao) của hình ảnh
                        shape_counts[shape] += 1
                except Exception as e:
                    print(f"Error processing image {img_path}: {e}")

    return shape_counts

# Ví dụ sử dụng
dataset_dir = "/content/dataset"
unique_shapes = check_unique_shapes(dataset_dir)

# In ra các shape và số lần xuất hiện
for shape, count in unique_shapes.items():
    print(f"Shape {shape} appears {count} times.")


👉 shape của những ảnh trong dataset là 224,224

In [ ]:
# xóa file metadata.json (chỉ có tác dụng khi upload data lên kaggle.)
!rm /content/dataset/dataset-metadata.json

### Pre-check

tải file .utils/ETL_functions.py từ thư mục đồ án

In [ ]:
from google.colab import files
files.upload()  # Thực hiện tải ETL_functions.py lên từ máy tính

In [ ]:
import ETL_functions as etl
from ETL_functions import *

### Mở bộ dữ liệu hoa

In [ ]:
train_path = "/content/dataset/DSP305x_01-A_VN-dataset/train"
test_path = "/content/dataset/DSP305x_01-A_VN-dataset/test"
val_path = "/content/dataset/DSP305x_01-A_VN-dataset/val"

In [ ]:
train= etl.load_dataset(train_path)
test = etl.load_dataset(test_path)
val = etl.load_dataset(val_path)
train

### Tạo model

#### thiết lập mixed_precision


In [ ]:
# Enable mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")
mixed_precision.global_policy()

In [ ]:
train.shape, test.shape, val.shape

#### Tạo tensorboard callbacks

In [ ]:
import tensorflow as tf
import datetime
from tensorflow.keras.applications import EfficientNetB2, InceptionV3, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, Flatten, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt
import os

In [ ]:
# Thiết lập thư mục cho TensorBoard logs
log_dir = "logs/fit"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Thiết lập checkpoint để lưu trọng số mô hình tốt nhất
checkpoint_dir = './model_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

xception_checkpoint_path = os.path.join(checkpoint_dir, 'xception_best_weights.h5')
inception_checkpoint_path = os.path.join(checkpoint_dir, 'inception_best_weights.h5')
vgg16_checkpoint_path = os.path.join(checkpoint_dir, 'vgg16_best_weights.h5')

xception_checkpoint = ModelCheckpoint(filepath=xception_checkpoint_path, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
inception_checkpoint = ModelCheckpoint(filepath=inception_checkpoint_path, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
vgg16_checkpoint = ModelCheckpoint(filepath=vgg16_checkpoint_path, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)


buffer

#### ImageDataGenerator : load ảnh

In [ ]:
#
datagen = ImageDataGenerator(
    rescale=1.0/255,
)

In [ ]:
# Create iterators for train, validation, and test data
train_generator = datagen.flow_from_dataframe(train, x_col='ImgPath', y_col='Class', target_size=(224, 224),batch_size=8)
val_generator = datagen.flow_from_dataframe(val, x_col='ImgPath', y_col='Class', target_size=(224, 224), batch_size = 8)
test_generator = datagen.flow_from_dataframe(test, x_col='ImgPath', y_col='Class', target_size=(224, 224), batch_size=8)


In [ ]:
for data_batch, labels_batch in train_generator:
    print("Data batch shape:", data_batch.shape)
    print("Labels batch shape:", labels_batch.shape)
    break

#### Build model

In [ ]:
def build_model(base_model, num_classes):
    # Freeze the base model
    base_model.trainable = False

    # Define the input layer
    inputs = Input(shape=(224, 224, 3))

    # Pass the input through the base model
    x = base_model(inputs, training=False)

    # Global Average Pooling for downsampling the feature map
    x = GlobalAveragePooling2D()(x)

    # Flatten the output for the fully connected layers
    x = Flatten()(x)

    # Add a fully connected dense layer with ReLU activation
    x = Dense(128, activation="relu")(x)

    # Add a dropout layer for regularization
    x = Dropout(0.5)(x)

    # Final output layer with softmax activation for multi-class classification
    outputs = Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = Model(inputs, outputs)
    return model


#### Compile model  

In [ ]:
# Number of classes (e.g., 10 classes for 10 flower types)
num_classes = 10

# Initialize the pre-trained models
vgg16_model = build_model(VGG16(weights="imagenet", include_top=False), num_classes)
xception_model = build_model(EfficientNetB2(weights="imagenet", include_top=False), num_classes)
inception_model = build_model(InceptionV3(weights="imagenet", include_top=False), num_classes)

# Compile the models with optimizer, loss, and metrics
vgg16_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
xception_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
inception_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
vgg16_model.summary()

In [ ]:
print("Training VGG16 model...")
history_vgg = vgg16_model.fit(train_generator,
                              epochs=10,
                              validation_data=val_generator,
                              callbacks=[vgg16_checkpoint])

In [ ]:
xception_model.summary()

In [ ]:
print("Training InceptionV3 model...")

history_xception = xception_model.fit(train_generator,
                                  epochs=10,
                                  validation_data=val_generator,
                                  callbacks=[xception_checkpoint])

In [ ]:
inception_model.summary()

In [ ]:
# Train the models
print("Training EfficientNetB0 model...")

history_inception = inception_model.fit(train_generator, epochs=10,
                                     validation_data=val_generator,
                                     callbacks=[inception_checkpoint])


### So sánh và chọn model tốt nhất cho bước tiếp theo

In [ ]:
import matplotlib.pyplot as plt

def plot_history(histories, titles):
    for i, history in enumerate(histories):
        plt.figure(figsize=(12, 4))
        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='train_accuracy')
        plt.plot(history.history['val_accuracy'], label='val_accuracy')
        plt.title(f'{titles[i]} - Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='train_loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.title(f'{titles[i]} - Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.show()

# Plot histories
plot_history([history_inception, history_vgg, history_xception],
             ['InceptionV3', 'VGG16', 'Xception_model'])

## Load model để fine-tune

In [ ]:
from tensorflow.keras.models import load_model

# Thay 'path_to_model' bằng đường dẫn chính xác đến file mô hình đã lưu
model_path = '/content/drive/MyDrive/Funix_Capstione_model/xception_finetuned_model.h5'
model = load_model(model_path)


In [ ]:
# In ra tên của mỗi lớp và trạng thái có thể train (trainable) của chúng
for layer in model.layers:
    print(f'Layer Name: {layer.name}, Trainable: {layer.trainable}')

### Mở thêm vài layer để train

In [ ]:
# Giả sử Xception là một lớp cơ sở trong mô hình của bạn
xception_base = model.get_layer('xception')
# Mở khóa các lớp trong Block13 và Block14
for layer in xception_base.layers:
    if 'block13' in layer.name or 'block14' in layer.name:
        layer.trainable = True
    else:
        layer.trainable = False  # Giữ nguyên các lớp khác không thay đổi


In [ ]:
# In ra trạng thái của mỗi lớp
for layer in model.layers:
    print(f'Layer: {layer.name}, Trainable: {layer.trainable}')

### Tạo model mới

In [ ]:
# Thêm BatchNormalization và Dropout vào các lớp đã mở khóa
x = xception_model.get_layer('flatten_2').output
x = BatchNormalization()(x)
x = Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.5)(x)
outputs = Dense(train_generator.num_classes, activation='softmax')(x)

# Tạo mô hình mới với các lớp đã được fine-tune
fine_tuned_model = Model(inputs=xception_model.input, outputs=outputs)

# Biên dịch lại mô hình
fine_tuned_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])


### Thiết lập checkpoint và callback cho finetune model

In [ ]:
checkpoint_dir = './xception_finetuned.keras'
os.makedirs(checkpoint_dir, exist_ok=True)

fine_tune_checkpoint = ModelCheckpoint(filepath=os.path.join(checkpoint_dir, 'xception_finetune_best_weights.weights.h5'),
                                       save_weights_only=True,
                                       monitor='val_loss',
                                       mode='min',
                                       save_best_only=True,
                                       verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

tensorboard_callback = TensorBoard(log_dir="logs/fit_finetune", histogram_freq=1)


### Data augmentation

In [ ]:
finetune_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
finetune_train_generator = finetune_datagen.flow_from_dataframe(train, x_col='ImgPath', y_col='Class', target_size=(224, 224),batch_size=8)

### Fit model

In [ ]:
fine_tuned_model.summary()

#### tính toán steps

In [ ]:
num_train_samples = int(20371 * 0.8)  # 80% dữ liệu cho huấn luyện
num_val_samples = int(20371 * 0.05)    # 20% dữ liệu cho validation

batch_size = 16

steps_per_epoch = num_train_samples // batch_size
validation_steps = num_val_samples // batch_size

In [ ]:
# Huấn luyện mô hình
fine_tuned_history = fine_tuned_model.fit(finetune_train_generator,
                                          epochs=20,
                                          steps_per_epoch=steps_per_epoch,
                                          validation_data=val_generator,
                                          validation_steps=validation_steps,
                                          callbacks=[fine_tune_checkpoint, early_stopping, tensorboard_callback])


In [ ]:
# đánh giá mô hình
test_loss, test_accuracy = fine_tuned_model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
asdsadsadsadcascsacd

In [ ]:
model.save("drive/My Drive/")

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

def extract_features(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

# Trích xuất đặc trưng cho tất cả ảnh
image_paths = [os.path.join('data_directory', img) for img in os.listdir('data_directory')]
features_list = [extract_features(img_path, model) for img_path in image_paths]
features_array = np.array(features_list)